In [ ]:
import dataloader
import model
import time
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import json
import torch
from torch.cuda.amp import autocast

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Funciones auxiliares para el dataset
def crear_csv():
    df = pd.read_csv('samples.csv')
    # Vector con todos los posibles identificadores 
    labels = []
    for n in range(10):
        labels.append('T00' + str(n))
    for n in range(10,100):
        labels.append('T0' + str(n))
    for n in range(100,201):
        labels.append('T' + str(n))
    label = np.array(labels)
    np.random.shuffle(label)
    numeros = np
    nd=201
    # Creamos el csv que guarde todos los posibles identificadores 
    d = {'index': list(range(0,nd )), 'mid': label, 'display_name':["''"]*nd}
    df = pd.DataFrame(data=d)
    df.to_csv('class_labels_indices.csv', index = False)

def crear_json():
    directory = 'canciones'
    df = pd.read_csv('samples.csv')
    diccionarios = []
    # Iteramos sobre archivos en ./canciones
    for filename in os.listdir(directory):
        direccion = os.path.join(directory, filename)
        if os.path.isfile(direccion):
            # Quitamos extensión
            original = filename.replace(".flac", "")
            # Determinamos qué samples contiene cada canción según samples.csv
            etiquetas = [*set([str(df['original_track_id'][i]) for i in list(df.index[df['sample_track_id'] == original])])]
            if len(etiquetas) != 0:
                etiquetas = etiquetas[0]
            else:
                # Placeholder si una canción no contiene samples
                etiquetas = 'T000'
            diccionario = {
            "wav": direccion,
            "labels": etiquetas
            }
            diccionarios.append(diccionario)
            
    data = {
        "data":diccionarios
    }
    json_object = json.dumps(data, indent=4)
    # Creamos json del dataset
    with open("train_data.json", "w") as outfile:
        outfile.write(json_object)

In [ ]:
# Creamos archivos auxiliares
crear_csv()
crear_json()

In [ ]:
# Entrenamiento
def train(model, epochs,data_loader,criterion,optimizer,cuda=False):
        x = np.arange(1, epochs + 1)
        y = np.empty(epochs)
        start = time.time()
        
        for epoch in range(epochs):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                if (cuda == True):
                    inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with autocast():
                    outputs = model.forward(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            y[epoch] = running_loss
            
        end = time.time()
        print("El entrenamiento tomó " + str(end - start) + " segundos.")
        # Grafica loss
        plt.plot(x, y)
        plt.xlabel('Número de epochs')
        plt.ylabel('Error')
        plt.show

In [ ]:
# Cargamos datos de entrenamiento
labels = 'class_labels_indices.csv'
data = dataloader.AudiosetDataset('train_data.json', label_csv = labels)
train_loader = torch.utils.data.DataLoader(data,batch_size=8,
                                          shuffle=True, num_workers=2)

In [ ]:
# Creamos y entrenamos modelo
modelo_audio = model.ASTModel(class_n = 201)
modelo_audio = modelo_audio.to(device)
entrenables = [p for p in modelo_audio.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(entrenables, 0.001, weight_decay=5e-7, betas=(0.95, 0.999))
criterio = torch.nn.CrossEntropyLoss()
#Entrenamos la red durante 50 pasos, con entropia cruzada y el optimizador ADAM
train(modelo_audio, 2,train_loader,criterio,optimizer,cuda=True)
# Guardar modelo
PATH = './modelo_audio.pth'
torch.save(modelo_audio.state_dict(), PATH)

In [ ]:
# Recomiendo ejecutar esto en lugar del entrenamiento
PATH = './modelo_audio.pth'
modelo_audio = model.ASTModel(class_n = 201)
modelo_audio.load_state_dict(torch.load(PATH))
modelo_audio = modelo_audio.to(device)

In [ ]:
# Funciones para evaluación
def contarCorrectas(net,batch,labels,func=None):
    '''Dado un batch y sus etiquetas, cuenta el numero de respuestas
    correctas de una red, el parametro func aplica una modificacion al 
    tensor que contiene los datos'''
    salidas=net(batch)
    cantidadCorrectas = 0
    for (output, label) in zip(salidas, labels):
        if torch.argmax(output) == torch.argmax(label):
            cantidadCorrectas = cantidadCorrectas + 1
    return cantidadCorrectas
    
def calcularPrecisionGlobal(net,data_loader,batch_size,cuda=False):
    '''Calcula la precision de una red dado un data_loader,
    recive una funcion que transforma los datos en caso de ser necesario'''
    correctas=0
    for (images,labels) in data_loader:
        if(cuda and torch.cuda.is_available()):
            images=images.cuda()
            labels=labels.cuda()
        correctas+=contarCorrectas(net,images,labels)
    return (100*correctas)/(len(data_loader)*batch_size) 

In [ ]:
precision = calcularPrecisionGlobal(modelo_audio,train_loader,8, cuda = True)
print("Precision del modelo: %.4f%%"%(precision))